In [10]:
import pandas as pd
from os import rename
from datetime import datetime
from all_p import ClusterFile

In [11]:
class Watch:
    table = None
    def_name = '~/watch/last.csv'
    def_dir = '~/watch/'
    states = ('ne', 'cl', 'grizym')
    def __new__(obj):
        if not hasattr(obj, 'instance'):
            obj.instance=super(Watch, obj).__new__();
        return obj.instance;
    
    def __init__(self):
        table = pd.read_csv(def_name, index_col='index')

    def name_backup():
        time = datetime.now()
        return 'back%d.%d_%d:%d:%d.csv' % (now.month, now.day, now.hour, now.minute, now.second)

    def __del__(self):
        rename(def_name, join(def_dir, name_backup()))
        self.table.to_csv(def_name)
        
    def get_patch_idx(self, id_list, id_patch):
        idx = self.table[self.table['id_list'] == id_list][self.table['id_patch'] == id_patch].index[0]
        if len(idx) > 0:
            return idx[0]
        return None
    
    def add_patch(self, cf, idx, i=None, id_list=None, from_list=True):
        if from_list:
            self.table['id_list'].loc[idx] = id_list
            self.table['id_patch'].loc[i] = i
        else:
            self.table['state'] = cf.params['state']
            
        self.table['ra'].loc[i] = cf.params['ra']
        self.table['dec'].loc[i] = cf.params['dec']
        self.table['inpix'].loc[i] = cf.params['inpix']
    
    def update_state(self, cf, idx):
        new_state = cf.params['state']
        if new_state in self.states:
            new_state_i = states.index(new_state)
            old_state_i = states.index(self.table['state'].iloc[idx])
            if new_state_i > old_state_i:
                self.table['state'].loc[idx] = new_state
                
    def update(self, fname, data=None, vrb=False):
        cf = ClusterFile(fname)
        if cf.params['typ'] == 'cen':
            if data is None:
                return
            id_list = cf.params['id_list']
            for i in range(data.shape[0]):
                idx = self.get_patch(id_list, idx, i, id_list)
                if idx is None:
                    idx = self.table.shape[0]
                    self.add_patch(cf, idx, i, id_list)
        elif cf.params['typ'] == 'data':
            if not (data is None):
                print('I dont need data for patch!')
            idx = self.get_patch_idx(cf.params['id_list'], cf.params['id_patch'])
            if idx is None:
                idx = self.table.shape[0]
                self.add_patch(cf, idx, from_list=False)
            